In [60]:
import pandas as pd
import numpy as np
np.random.seed(2020)
from pprint import pprint

# used for pre-processing and modeling
import gensim
from gensim import models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from gensim.models.wrappers import LdaMallet

# used for natural language processing {NLTK: Natural Language Tool-Kit} 
from nltk.stem import WordNetLemmatizer, SnowballStemmer, LancasterStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')
import spacy

# Visualization tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.spatial import distance

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/siddharth/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [61]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [92]:
# Load dataset
plots = pd.read_csv("wiki_movie_plots_deduped.csv")
plots

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
...,...,...,...,...,...,...,...,...
34881,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ..."
34882,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the..."
34883,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i..."
34884,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...


## Model 1:

1. WordNetLemmatizer
2. TF-IDF
3. LDAMulticore  

In [3]:
# lemmatizer
lemmatizer = WordNetLemmatizer()

# spacy model
nlp = spacy.load("en")

In [4]:
# function to lemmatize a given word
def lemmatize(text):
    return lemmatizer.lemmatize(text, pos = "n")

# function to preprocess the text
def preprocess(text):
    result = []
    for word in gensim.utils.simple_preprocess(text):
        if word not in gensim.parsing.preprocessing.STOPWORDS and len(word) > 3:
            l = lemmatize(word)
            if len(l) > 3:
                result.append(l)
    return result

In [5]:
plots_processed = plots['Plot'].map(preprocess)
plots_processed

0        [bartender, working, saloon, serving, drink, c...
1        [moon, painted, smiling, face, hang, park, nig...
2        [film, minute, long, composed, shot, girl, sit...
3        [lasting, second, consisting, shot, shot, wood...
4        [earliest, known, adaptation, classic, fairyta...
5        [alice, follows, large, white, rabbit, rabbit,...
6        [film, open, bandit, breaking, railroad, teleg...
7        [film, family, suburb, hoping, quiet, life, th...
8        [opening, scene, show, interior, robber, wall,...
9        [scene, introduced, line, poem, santa, claus, ...
10       [rarebit, fiend, gorge, welsh, rarebit, restau...
11       [film, feature, train, traveling, rockies, hol...
12       [irish, villager, kathleen, tenant, captain, c...
13       [boone, daughter, befriends, indian, maiden, b...
14       [heading, baseball, game, nearby, ballpark, sp...
15       [plot, black, woman, going, dentist, toothache...
16       [beautiful, summer, father, mother, daughter, .

In [6]:
# Dictionary for LDA model
plots_dict = Dictionary(plots_processed)
len(plots_dict)

118500

In [7]:
# Filter dictonary
plots_dict.filter_extremes(no_below = 2000, no_above = 0.5) #948
len(plots_dict)

303

In [8]:
# function that provides frequency of words in the dictionary
def freq_list(dict):
    l = []
    for k, v in dict.iteritems():
        l.append(v)
    
    freq_dict = dict.dfs
    freq_df = pd.DataFrame.from_dict(freq_dict, orient = 'index', columns = ['Frequency']).reset_index()
    freq_df = freq_df.sort_values(by = 'index').reset_index()
    freq_df.drop(columns = ['level_0'], inplace = True)
    freq_df.insert(1, 'word', l)
    
    freq = freq_df.sort_values(by = 'Frequency')
    
    return freq

In [9]:
# Bag of words and TFIDF
bag_of_words = [plots_dict.doc2bow(plot) for plot in plots_processed]
tfidf = models.TfidfModel(bag_of_words)
corpus_tfidf = tfidf[bag_of_words]

In [10]:
# LDA Multicore model using the tf-idf corpus
lda_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=plots_dict, passes=10,
                                       workers = 7, random_state = 2020)

for idx, topic in lda_tfidf.print_topics(-1):
    print('Topic: {} \nWords: {}\n'.format(idx, topic))

Topic: 0 
Words: 0.049*"gang" + 0.021*"police" + 0.016*"money" + 0.015*"leader" + 0.013*"kill" + 0.013*"brother" + 0.013*"criminal" + 0.012*"member" + 0.010*"fight" + 0.009*"escape"

Topic: 1 
Words: 0.050*"school" + 0.036*"student" + 0.025*"john" + 0.018*"high" + 0.012*"father" + 0.012*"girl" + 0.012*"college" + 0.012*"mother" + 0.011*"year" + 0.011*"friend"

Topic: 2 
Words: 0.063*"village" + 0.021*"land" + 0.019*"power" + 0.016*"people" + 0.011*"story" + 0.010*"form" + 0.010*"father" + 0.010*"life" + 0.009*"save" + 0.009*"love"

Topic: 3 
Words: 0.013*"kill" + 0.010*"escape" + 0.010*"body" + 0.010*"group" + 0.010*"attack" + 0.008*"killed" + 0.008*"attempt" + 0.008*"house" + 0.008*"head" + 0.007*"water"

Topic: 4 
Words: 0.030*"army" + 0.020*"american" + 0.020*"town" + 0.016*"battle" + 0.012*"country" + 0.011*"force" + 0.010*"officer" + 0.010*"state" + 0.010*"order" + 0.010*"attack"

Topic: 5 
Words: 0.029*"story" + 0.029*"love" + 0.018*"college" + 0.017*"life" + 0.016*"girl" + 0.016

In [11]:
# Model measure of performance
cm = CoherenceModel(model = lda_tfidf, texts = plots_processed, dictionary = plots_dict)
cm.get_coherence()

0.39250017417614014

In [12]:
# Visualization
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_tfidf, corpus_tfidf, plots_dict)
vis

/home/ss3zg/.local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.009028  0.000930       1        1  18.924730
3     -0.052879 -0.010863       2        1  17.057129
7      0.057789  0.044649       3        1  12.720376
8     -0.037441  0.030543       4        1   9.345959
4     -0.050381 -0.018002       5        1   8.773159
5      0.082745 -0.001891       6        1   7.530401
0     -0.046443  0.028834       7        1   7.231372
6      0.011793 -0.062344       8        1   6.725647
1      0.027429 -0.024598       9        1   6.685310
2     -0.001640  0.012741      10        1   5.005919, topic_info=     Category         Freq      Term        Total  loglift  logprob
term                                                               
124   Default   998.000000   village   998.000000  30.0000  30.0000
89    Default   780.000000      gang   780.000000  29.0000  29.0000
299   Default   917.000000    school   917.000000  28.0000  28.0000
300   Default   669.000000   student   669.000000  27.0000  27.0000
19    Default  1389.000000      film  1389.000000  26.0000  26.0000
214   Default   607.000000      army   607.000000  25.0000  25.0000
213   Default   665.000000      team   665.000000  24.0000  24.0000
100   Default  1263.000000    police  1263.000000  23.0000  23.0000
186   Default   804.000000    murder   804.000000  22.0000  22.0000
172   Default  1224.000000     story  1224.000000  21.0000  21.0000
176   Default  1472.000000      love  1472.000000  20.0000  20.0000
165   Default   615.000000      john   615.000000  19.0000  19.0000
270   Default   694.000000     movie   694.000000  18.0000  18.0000
212   Default   578.000000   college   578.000000  17.0000  17.0000
195   Default   502.000000      land   502.000000  16.0000  16.0000
56    Default   629.000000      play   629.000000  15.0000  15.0000
236   Default   508.000000     power   508.000000  14.0000  14.0000
122   Default   773.000000  marriage   773.000000  13.0000  13.0000
151   Default   548.000000  american   548.000000  12.0000  12.0000
182   Default   503.000000     crime   503.000000  11.0000  11.0000
87    Default   460.000000  criminal   460.000000  10.0000  10.0000
48    Default   509.000000     dream   509.000000   9.0000   9.0000
269   Default   543.000000      case   543.000000   8.0000   8.0000
137   Default   763.000000   officer   763.000000   7.0000   7.0000
190   Default   509.000000    battle   509.000000   6.0000   6.0000
290   Default   484.000000      high   484.000000   5.0000   5.0000
118   Default   747.000000     world   747.000000   4.0000   4.0000
94    Default   463.000000    leader   463.000000   3.0000   3.0000
161   Default   781.000000      town   781.000000   2.0000   2.0000
81    Default  1281.000000      life  1281.000000   1.0000   1.0000
...       ...          ...       ...          ...      ...      ...
241   Topic10    43.475605    living   491.290466   0.5697  -5.2582
155   Topic10    46.646633      rest   529.421936   0.5654  -5.1878
118   Topic10    65.213066     world   747.188721   0.5559  -4.8528
157   Topic10    32.933247      wish   378.020264   0.5541  -5.5360
71    Topic10    57.783386     local   672.391296   0.5404  -4.9737
15    Topic10    41.631363      past   488.410583   0.5322  -5.3016
129   Topic10    22.963614    nearby   274.726257   0.5127  -5.8965
144   Topic10    39.269371   revenge   478.887207   0.4935  -5.3600
190   Topic10    41.751461    battle   509.406982   0.4930  -5.2987
169   Topic10    31.455967      true   384.170197   0.4921  -5.5819
172   Topic10    91.252327     story  1224.781860   0.3977  -4.5168
18    Topic10    69.983887     young  1002.613037   0.3324  -4.7822
81    Topic10    82.769737      life  1281.044678   0.2552  -4.6144
110   Topic10    56.713047     fight   793.584839   0.3560  -4.9924
121   Topic10    86.781670    father  1475.227417   0.1614  -4.5670
150   Topic10    64.453590      year  1015.73

### Model 2:

using spacy for lemmatizing

In [5]:
def lemmatize_nlp(doc, pos = ['NOUN', 'ADJ', 'ADV', 'VERB']):
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-' and token.pos_ in pos]
    doc = u' '.join(doc)
    return nlp.make_doc(doc)

def remove_stopwords(doc):
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

nlp.add_pipe(lemmatize_nlp, name='lemmatizer', after='ner')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

In [6]:
plots_processed_spacy = plots['Plot'].str.lower().map(nlp)

In [7]:
# Dictionary for LDA model
plots_dict_spacy = Dictionary(plots_processed_spacy)
len(plots_dict_spacy)

71514

In [8]:
# Filter dictonary
plots_dict_spacy.filter_extremes(no_below = 1500, no_above = 0.5) # 200
len(plots_dict_spacy)

479

In [9]:
# Bag of words and TFIDF
bow = [plots_dict_spacy.doc2bow(plot) for plot in plots_processed_spacy]
tfidf_mod = models.TfidfModel(bow)
corpus = tfidf_mod[bow]

In [18]:
# LDA Multicore model using the tf-idf corpus
lda_mod = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=plots_dict_spacy, passes=10, workers=7, 
                                       random_state = 2020)

for idx, topic in lda_mod.print_topics(-1):
    print('Topic: {} \nWords: {}\n'.format(idx, topic))

Topic: 0 
Words: 0.026*"film" + 0.021*"school" + 0.020*"student" + 0.018*"story" + 0.014*"play" + 0.012*"life" + 0.012*"team" + 0.012*"movie" + 0.011*"character" + 0.010*"teacher"

Topic: 1 
Words: 0.010*"car" + 0.008*"house" + 0.008*"tell" + 0.008*"kill" + 0.007*"police" + 0.007*"room" + 0.007*"find" + 0.006*"apartment" + 0.006*"leave" + 0.006*"man"

Topic: 2 
Words: 0.012*"ship" + 0.010*"destroy" + 0.010*"kill" + 0.009*"human" + 0.009*"attack" + 0.009*"power" + 0.009*"use" + 0.008*"escape" + 0.007*"group" + 0.007*"world"

Topic: 3 
Words: 0.010*"company" + 0.010*"money" + 0.009*"job" + 0.009*"win" + 0.007*"work" + 0.007*"business" + 0.007*"play" + 0.007*"big" + 0.007*"star" + 0.006*"dance"

Topic: 4 
Words: 0.010*"relationship" + 0.008*"mother" + 0.008*"love" + 0.008*"letter" + 0.008*"friend" + 0.007*"write" + 0.007*"life" + 0.007*"tell" + 0.007*"book" + 0.007*"day"

Topic: 5 
Words: 0.019*"police" + 0.019*"murder" + 0.013*"kill" + 0.012*"case" + 0.010*"officer" + 0.010*"jail" + 0.01

In [19]:
cm_spacy = CoherenceModel(model = lda_mod, texts = plots_processed_spacy, dictionary = plots_dict_spacy)
cm_spacy.get_coherence()

0.43750702086131527

In [20]:
# Visualization
pyLDAvis.enable_notebook()
vis_spacy = pyLDAvis.gensim.prepare(lda_mod, corpus, plots_dict_spacy)
vis_spacy

/home/ss3zg/.local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.015184  0.025717       1        1  17.457428
2      0.066862 -0.014235       2        1  12.888718
8     -0.064452  0.020885       3        1  12.529513
4     -0.048438 -0.002996       4        1  12.083308
3     -0.017234 -0.011769       5        1  10.722033
5      0.012467  0.032138       6        1   8.916404
0     -0.030363 -0.078036       7        1   6.776922
9      0.041535  0.010110       8        1   6.755426
6      0.051491 -0.014495       9        1   6.548120
7     -0.027052  0.032683      10        1   5.322139, topic_info=     Category         Freq       Term        Total  loglift  logprob
term                                                                
134   Default   651.000000       gang   651.000000  30.0000  30.0000
66    Default   466.000000       baby   466.000000  29.0000  29.0000
274   Default   391.000000    soldier   391.000000  28.0000  28.0000
219   Default   493.000000        war   493.000000  27.0000  27.0000
29    Default  1072.000000       film  1072.000000  26.0000  26.0000
146   Default   991.000000     police   991.000000  25.0000  25.0000
468   Default   561.000000    student   561.000000  24.0000  24.0000
467   Default   737.000000     school   737.000000  23.0000  23.0000
67    Default   835.000000      child   835.000000  22.0000  22.0000
379   Default   345.000000       army   345.000000  21.0000  21.0000
269   Default   435.000000     battle   435.000000  20.0000  20.0000
233   Default   751.000000     murder   751.000000  19.0000  19.0000
251   Default   954.000000      story   954.000000  18.0000  18.0000
58    Default  1307.000000       kill  1307.000000  17.0000  17.0000
189   Default   691.000000    husband   691.000000  16.0000  16.0000
131   Default   367.000000   criminal   367.000000  15.0000  15.0000
182   Default   817.000000    village   817.000000  14.0000  14.0000
321   Default   410.000000       ship   410.000000  13.0000  13.0000
310   Default   518.000000       team   518.000000  12.0000  12.0000
180   Default   629.000000   marriage   629.000000  11.0000  11.0000
195   Default   574.000000    officer   574.000000  10.0000  10.0000
478   Default   357.000000     killer   357.000000   9.0000   9.0000
350   Default   474.000000     doctor   474.000000   8.0000   8.0000
351   Default   383.000000       drug   383.000000   7.0000   7.0000
79    Default   706.000000       play   706.000000   6.0000   6.0000
26    Default   905.000000      woman   905.000000   5.0000   5.0000
237   Default   810.000000      marry   810.000000   4.0000   4.0000
254   Default  1239.000000       love  1239.000000   3.0000   3.0000
261   Default   379.000000      crime   379.000000   2.0000   2.0000
141   Default   377.000000     leader   377.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
284   Topic10    42.673496       trip   291.512329   1.0118  -5.5073
119   Topic10   130.395432       wife   909.206604   0.9913  -4.3903
27    Topic10   124.377365      young   885.717651   0.9702  -4.4375
199   Topic10    35.515125  beautiful   269.213898   0.9077  -5.6909
429   Topic10    29.530809    deliver   227.987854   0.8894  -5.8754
378   Topic10    29.178064      treat   227.280823   0.8805  -5.8874
266   Topic10    54.086090      small   426.742523   0.8677  -5.2703
17    Topic10    59.152519     couple   470.518646   0.8596  -5.1807
259   Topic10    34.555332     suffer   287.578613   0.8144  -5.7183
176   Topic10    53.456661     travel   445.500977   0.8130  -5.2820
153   Topic10    53.738758     search   449.213837   0.8099  -5.2767
59    Topic10   115.736313     mother   989.133179   0.7878  -4.5095
241   Topic10    35.797520     little   314.465729   0.7603  -5.6830
242   Topic10    82.340141        old   740.910767   0.7363  -4.8500
339   Topic10    37.126404    country   345.515564   0.7026  -5.64

### Model 3:
Using Bigrams

In [21]:
bigrams = gensim.models.Phrases(plots_processed, min_count=10, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigrams)

In [22]:
def create_bigrams(text):
    return bigram_mod[text]

In [23]:
bigrams_processed = plots_processed.map(create_bigrams)
bigrams_spacy = plots_processed_spacy.map(create_bigrams)

In [24]:
# Dictionary for LDA model
bigrams_dict = Dictionary(bigrams_processed)
print(len(bigrams_dict))

bigrams_dict_spacy = Dictionary(bigrams_spacy)
print(len(bigrams_dict_spacy))

122071
73531


In [25]:
# Filter dictonary
bigrams_dict.filter_extremes(no_below = 1700, no_above = 0.5)
print(len(bigrams_dict))

bigrams_dict_spacy.filter_extremes(no_below = 2000, no_above = 0.5)
print(len(bigrams_dict_spacy))

359
329


In [26]:
f = freq_list(bigrams_dict)
f_spacy = freq_list(bigrams_dict_spacy)

#### Bigram model with general lemmatizer

In [27]:
# Bag of words and TFIDF
bow_bigrams = [bigrams_dict.doc2bow(plot) for plot in bigrams_processed]
tfidf_bigrams = models.TfidfModel(bow_bigrams)
corpus_bigrams = tfidf_bigrams[bow_bigrams]

In [28]:
# LDA Multicore model using the tf-idf corpus
lda_bigram = gensim.models.LdaMulticore(corpus_bigrams, num_topics=10, id2word=bigrams_dict, passes=10, workers = 7, 
                                        random_state = 2020)

for idx, topic in lda_bigram.print_topics(-1):
    print('Topic: {} \nWords: {}\n'.format(idx, topic))

Topic: 0 
Words: 0.025*"army" + 0.021*"village" + 0.021*"soldier" + 0.016*"battle" + 0.014*"american" + 0.010*"attack" + 0.010*"force" + 0.009*"fight" + 0.008*"people" + 0.008*"kill"

Topic: 1 
Words: 0.040*"team" + 0.024*"game" + 0.021*"agent" + 0.021*"captain" + 0.020*"mission" + 0.009*"officer" + 0.008*"play" + 0.008*"member" + 0.007*"secret" + 0.007*"kill"

Topic: 2 
Words: 0.032*"murder" + 0.018*"case" + 0.016*"woman" + 0.016*"husband" + 0.015*"murdered" + 0.015*"wife" + 0.014*"doctor" + 0.014*"police" + 0.012*"young" + 0.012*"plot"

Topic: 3 
Words: 0.011*"tell" + 0.010*"house" + 0.009*"home" + 0.009*"mother" + 0.008*"night" + 0.008*"apartment" + 0.008*"room" + 0.008*"father" + 0.007*"friend" + 0.007*"child"

Topic: 4 
Words: 0.020*"love" + 0.018*"village" + 0.018*"marriage" + 0.016*"father" + 0.015*"family" + 0.015*"marry" + 0.015*"daughter" + 0.014*"married" + 0.014*"sister" + 0.013*"brother"

Topic: 5 
Words: 0.061*"film" + 0.022*"star" + 0.022*"movie" + 0.021*"london" + 0.018

In [29]:
cm_bigrams = CoherenceModel(model = lda_bigram, texts = bigrams_processed, dictionary = bigrams_dict)
cm_bigrams.get_coherence()

0.3960279064232234

In [30]:
# Visualization
pyLDAvis.enable_notebook()
vis_bigram = pyLDAvis.gensim.prepare(lda_bigram, corpus_bigrams, bigrams_dict)
vis_bigram

/home/ss3zg/.local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.016177  0.032428       1        1  19.815926
4      0.068821  0.037480       2        1  13.957363
7     -0.047275  0.020546       3        1  12.535170
6     -0.031479  0.010798       4        1  12.117929
9      0.012559  0.013586       5        1   8.434177
0     -0.046866  0.008283       6        1   8.080917
8      0.076042 -0.016603       7        1   7.957297
2      0.010793 -0.023308       8        1   6.563347
1     -0.058330  0.001689       9        1   6.001185
5     -0.000440 -0.084900      10        1   4.536702, topic_info=     Category         Freq       Term        Total  loglift  logprob
term                                                                
19    Default  1349.000000       film  1349.000000  30.0000  30.0000
243   Default   660.000000       team   660.000000  29.0000  29.0000
207   Default   787.000000     murder   787.000000  28.0000  28.0000
137   Default   950.000000    village   950.000000  27.0000  27.0000
244   Default   496.000000       army   496.000000  26.0000  26.0000
97    Default   722.000000       gang   722.000000  25.0000  25.0000
142   Default   445.000000       game   445.000000  24.0000  24.0000
193   Default  1118.000000      story  1118.000000  23.0000  23.0000
313   Default   638.000000      movie   638.000000  22.0000  22.0000
215   Default   412.000000    soldier   412.000000  21.0000  21.0000
132   Default   418.000000    captain   418.000000  20.0000  20.0000
168   Default   601.000000    company   601.000000  19.0000  19.0000
242   Default   404.000000       star   404.000000  18.0000  18.0000
109   Default  1202.000000     police  1202.000000  17.0000  17.0000
24    Default   454.000000      agent   454.000000  16.0000  16.0000
357   Default   391.000000     london   391.000000  15.0000  15.0000
332   Default   417.000000    mission   417.000000  14.0000  14.0000
240   Default   504.000000    college   504.000000  13.0000  13.0000
353   Default   574.000000    student   574.000000  12.0000  12.0000
203   Default   476.000000      crime   476.000000  11.0000  11.0000
211   Default   439.000000     battle   439.000000  10.0000  10.0000
309   Default   527.000000       case   527.000000   9.0000   9.0000
197   Default  1362.000000       love  1362.000000   8.0000   8.0000
280   Default   385.000000  character   385.000000   7.0000   7.0000
135   Default   674.000000   marriage   674.000000   6.0000   6.0000
347   Default   345.000000   murdered   345.000000   5.0000   5.0000
153   Default   696.000000    officer   696.000000   4.0000   4.0000
143   Default   764.000000    husband   764.000000   3.0000   3.0000
84    Default  1209.000000       life  1209.000000   2.0000   2.0000
56    Default   583.000000       play   583.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
56    Topic10    87.190331       play   583.846985   1.1914  -4.4925
154   Topic10    55.942905       plot   426.825073   1.0609  -4.9363
225   Topic10    73.568161       lead   582.289978   1.0242  -4.6624
39    Topic10    47.465801      known   386.116730   0.9968  -5.1006
193   Topic10   137.471115      story  1118.848022   0.9963  -4.0372
123   Topic10    57.219639       real   475.492767   0.9755  -4.9137
83    Topic10    33.895123    leading   306.205139   0.8920  -5.4373
130   Topic10    75.806274      world   703.057861   0.8657  -4.6324
161   Topic10    41.678246    revenge   404.909637   0.8193  -5.2306
268   Topic10    27.154310   struggle   289.179657   0.7275  -5.6591
2     Topic10    62.320026      group   744.081055   0.6131  -4.8283
102   Topic10    35.192616     leader   430.457001   0.5890  -5.3998
257   Topic10    33.974758     travel   415.801910   0.5884  -5.4350
236   Topic10    25.870687    witness   316.711945   0.5881  -5.7075
139   Topic10    33.524494  encounter   410.701080   0.5874  -5.44

#### Bigram model with spacy lemmatizer

In [31]:
# Bag of words and TFIDF
bow_bigrams_spacy = [bigrams_dict_spacy.doc2bow(plot) for plot in bigrams_spacy]
tfidf_bigrams_spacy = models.TfidfModel(bow_bigrams_spacy)
corpus_bigrams_spacy = tfidf_bigrams_spacy[bow_bigrams_spacy]

In [32]:
# LDA Multicore model using the tf-idf corpus
lda_bigram_mod = gensim.models.LdaMulticore(corpus_bigrams_spacy, num_topics=10, id2word=bigrams_dict_spacy, passes=10, workers = 7, 
                                        random_state = 2020)

for idx, topic in lda_bigram_mod.print_topics(-1):
    print('Topic: {} \nWords: {}\n'.format(idx, topic))

Topic: 0 
Words: 0.040*"student" + 0.035*"school" + 0.026*"college" + 0.024*"team" + 0.012*"girl" + 0.011*"friend" + 0.011*"win" + 0.010*"play" + 0.009*"love" + 0.009*"father"

Topic: 1 
Words: 0.018*"love" + 0.014*"wedding" + 0.013*"marry" + 0.012*"marriage" + 0.012*"friend" + 0.012*"parent" + 0.011*"family" + 0.011*"father" + 0.010*"meet" + 0.010*"life"

Topic: 2 
Words: 0.027*"village" + 0.020*"love" + 0.019*"marry" + 0.019*"son" + 0.017*"brother" + 0.017*"family" + 0.016*"father" + 0.016*"daughter" + 0.016*"sister" + 0.015*"marriage"

Topic: 3 
Words: 0.017*"money" + 0.008*"job" + 0.008*"pay" + 0.008*"win" + 0.008*"offer" + 0.008*"tell" + 0.007*"work" + 0.007*"sell" + 0.007*"steal" + 0.007*"buy"

Topic: 4 
Words: 0.017*"murder" + 0.015*"police" + 0.013*"kill" + 0.012*"body" + 0.011*"car" + 0.010*"case" + 0.009*"find" + 0.009*"house" + 0.009*"man" + 0.009*"dead"

Topic: 5 
Words: 0.036*"town" + 0.023*"boy" + 0.016*"small" + 0.014*"train" + 0.014*"local" + 0.014*"girl" + 0.014*"young

In [33]:
cm_bigrams_spacy = CoherenceModel(model = lda_bigram_mod, texts = bigrams_spacy, dictionary = bigrams_dict_spacy)
cm_bigrams_spacy.get_coherence()

0.4122192899496212

In [34]:
# Visualization
pyLDAvis.enable_notebook()
vis_bigram_spacy = pyLDAvis.gensim.prepare(lda_bigram_mod, corpus_bigrams_spacy, bigrams_dict_spacy)
vis_bigram_spacy

/home/ss3zg/.local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.064052 -0.016105       1        1  18.270996
4     -0.045853  0.027918       2        1  14.554734
3     -0.015822  0.016037       3        1  11.542914
6      0.019374  0.025941       4        1  10.756473
2      0.049824  0.010403       5        1   9.789348
1      0.051265  0.029327       6        1   9.377187
8     -0.035233 -0.005621       7        1   8.493004
7      0.029092 -0.080872       8        1   6.042281
0      0.014771  0.006007       9        1   5.782709
5     -0.003366 -0.013036      10        1   5.390355, topic_info=     Category         Freq      Term        Total  loglift  logprob
term                                                               
24    Default  1283.000000      film  1283.000000  30.0000  30.0000
325   Default   650.000000   student   650.000000  29.0000  29.0000
189   Default   762.000000      town   762.000000  28.0000  28.0000
324   Default   702.000000    school   702.000000  27.0000  27.0000
148   Default   895.000000   village   895.000000  26.0000  26.0000
200   Default  1174.000000     story  1174.000000  25.0000  25.0000
244   Default   600.000000      team   600.000000  24.0000  24.0000
242   Default   538.000000   college   538.000000  23.0000  23.0000
111   Default   694.000000      gang   694.000000  22.0000  22.0000
121   Default  1165.000000    police  1165.000000  21.0000  21.0000
162   Default   783.000000       boy   783.000000  20.0000  20.0000
314   Default   459.000000    prison   459.000000  19.0000  19.0000
188   Default   946.000000    murder   946.000000  18.0000  18.0000
203   Default  1444.000000      love  1444.000000  17.0000  17.0000
146   Default   725.000000  marriage   725.000000  16.0000  16.0000
63    Default   869.000000      play   869.000000  15.0000  15.0000
191   Default   968.000000     marry   968.000000  14.0000  14.0000
294   Default   626.000000     movie   626.000000  13.0000  13.0000
209   Default   447.000000     crime   447.000000  12.0000  12.0000
25    Default  1087.000000      girl  1087.000000  11.0000  11.0000
159   Default   448.000000      plot   448.000000  10.0000  10.0000
99    Default  1192.000000      life  1192.000000   9.0000   9.0000
153   Default   828.000000   husband   828.000000   8.0000   8.0000
187   Default   415.000000      jail   415.000000   7.0000   7.0000
213   Default   509.000000     small   509.000000   6.0000   6.0000
252   Default   449.000000   wedding   449.000000   5.0000   5.0000
94    Default   573.000000     train   573.000000   4.0000   4.0000
158   Default   667.000000   officer   667.000000   3.0000   3.0000
96    Default  1285.000000    family  1285.000000   2.0000   2.0000
253   Default   466.000000    affair   466.000000   1.0000   1.0000
...       ...          ...       ...          ...      ...      ...
230   Topic10    50.827908      land   501.272766   0.6319  -5.2639
125   Topic10    49.363369    search   510.073975   0.5852  -5.2932
101   Topic10    29.882242   abandon   312.738831   0.5725  -5.7951
143   Topic10    32.857258   wealthy   357.431213   0.5338  -5.7002
211   Topic10    76.022202      live   857.553589   0.4975  -4.8613
258   Topic10    34.609619   country   390.743469   0.4966  -5.6482
270   Topic10    30.463312   trouble   350.996033   0.4763  -5.7758
7     Topic10   110.657722       man  1303.823853   0.4539  -4.4859
44    Topic10    38.469810      grow   454.266052   0.4517  -5.5425
163   Topic10    31.487200       buy   372.146088   0.4509  -5.7428
12    Topic10    36.557091       big   434.247223   0.4458  -5.5935
90    Topic10    31.371895   station   384.112061   0.4155  -5.7465
28    Topic10    59.775429       run   785.013123   0.3455  -5.1018
21    Topic10    77.242805     woman  1045.978760   0.3148  -4.8454
52    Topic10    68.572647     child   951.342773   0.2906  -4.9645
36    Topic10    53.733105       set   708.52

### Optimizing number of topics for best model (model #2)

In [49]:
def model_eval(corpus, dictionary, texts, limit=30, start=2, step=2):
    coherence = []
    models = []
    for num_topics in range(start, limit, step):
        model = gensim.models.LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, passes=10, workers=7, 
                                       random_state = 2020)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary)
        models.append(model)
        coherence.append(coherencemodel.get_coherence())

    return models, coherence

In [ ]:
models_list, coherence_vals = model_eval(corpus=corpus, dictionary=plots_dict_spacy, 
                                         texts = plots_processed_spacy, limit=16, start=2, step=1)

In [ ]:
limit=16
start=2
step=1

x = range(start, limit, step)

plt.plot(x, coherence_vals)
plt.xlabel("Number of Topics")
plt.ylabel("Coherence score")
plt.legend(("C values"), loc='best')
plt.title("Optimizing no. of topics for model #2")
plt.show()

In [ ]:
coherence_vals

In [26]:
# LDA Multicore model using the tf-idf corpus
final_mod = gensim.models.LdaMulticore(corpus, num_topics=8, id2word=plots_dict_spacy, passes=10, workers=7, 
                                       random_state = 2020)

for idx, topic in final_mod.print_topics(-1):
    print('Topic: {} \nWords: {}\n'.format(idx, topic))

Topic: 0 
Words: 0.027*"film" + 0.020*"story" + 0.016*"school" + 0.016*"student" + 0.014*"play" + 0.013*"life" + 0.012*"movie" + 0.011*"character" + 0.010*"team" + 0.009*"role"

Topic: 1 
Words: 0.009*"car" + 0.008*"tell" + 0.008*"house" + 0.007*"room" + 0.007*"night" + 0.006*"leave" + 0.006*"find" + 0.006*"hotel" + 0.006*"apartment" + 0.006*"home"

Topic: 2 
Words: 0.010*"kill" + 0.009*"ship" + 0.009*"attack" + 0.008*"soldier" + 0.007*"battle" + 0.007*"escape" + 0.007*"destroy" + 0.007*"use" + 0.007*"war" + 0.007*"power"

Topic: 3 
Words: 0.028*"gang" + 0.014*"money" + 0.012*"town" + 0.010*"big" + 0.008*"win" + 0.008*"boss" + 0.007*"star" + 0.007*"play" + 0.006*"local" + 0.006*"job"

Topic: 4 
Words: 0.010*"love" + 0.009*"family" + 0.009*"mother" + 0.009*"father" + 0.009*"child" + 0.008*"marry" + 0.008*"friend" + 0.007*"life" + 0.007*"parent" + 0.007*"relationship"

Topic: 5 
Words: 0.020*"village" + 0.016*"love" + 0.014*"marry" + 0.014*"son" + 0.013*"brother" + 0.013*"marriage" + 0.0

In [27]:
cm_final = CoherenceModel(model = final_mod, texts = plots_processed_spacy, dictionary = plots_dict_spacy)
cm_final.get_coherence()

0.44155065607560084

In [28]:
# Visualization
pyLDAvis.enable_notebook()
visual = pyLDAvis.gensim.prepare(final_mod, corpus, plots_dict_spacy)
visual

/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.059049 -0.023054       1        1  19.605307
1     -0.030214  0.006741       2        1  18.870325
4      0.034656  0.007037       3        1  16.213411
7     -0.050497  0.026318       4        1  12.688190
5      0.046136  0.033457       5        1  11.570615
3     -0.022636 -0.017055       6        1   8.352571
0      0.046985 -0.068277       7        1   8.050195
6      0.034619  0.034834       8        1   4.649384, topic_info=    Category         Freq     Term        Total  loglift  logprob
134  Default   724.000000     gang   724.000000  30.0000  30.0000
26   Default   938.000000    woman   938.000000  29.0000  29.0000
29   Default  1090.000000     film  1090.000000  28.0000  28.0000
182  Default   748.000000  village   748.000000  27.0000  27.0000
251  Default   995.000000    story   995.000000  26.0000  26.0000
..       ...          ...      ...          ...      ...      ...
14    Topic8    54.961285     work   834.024292   0.3488  -5.1191
54    Topic8    57.504375     find  1040.962402   0.1724  -5.0738
29    Topic8    53.783634     film  1090.895630   0.0587  -5.1407
194   Topic8    50.712780     meet   855.203247   0.2433  -5.1995
233   Topic8    49.255749   murder   829.820923   0.2443  -5.2287

[395 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
424       1  0.079048  accident
424       2  0.193794  accident
424       3  0.198894  accident
424       4  0.122396  accident
424       5  0.160645  accident
...     ...       ...       ...
27        4  0.091731     young
27        5  0.125993     young
27        6  0.075153     young
27        7  0.114941     young
27        8  0.175727     young

[2143 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 5, 8, 6, 4, 1, 7])

In [32]:
def assign_topic_dist(text, model = final_mod, dictionary = plots_dict_spacy):
    bag_of_words = dictionary.doc2bow(nlp(text))
    
    prob_list = []
    indlist = []
    
    for i, score in model[bag_of_words]:
        indlist.append(i)
        prob_list.append(round(score,4))
    
    n_topics = model.print_topics(-1)[-1][0] + 1
    diff = list(set(list(range(0,n_topics))).difference(indlist))

    while len(prob_list) < 8:
        for pos in diff:
            prob_list.insert(pos, 0.0)
        
    return prob_list



def show_topic_dist(text, model = final_mod, dictionary = plots_dict_spacy):
    bag_of_words = dictionary.doc2bow(nlp(text))
    
    for i, score in sorted(model[bag_of_words], key = lambda x: -1*x[1]):
        print("\nScore: {}\nTopic {}: {}".format(round(score,4), i, model.print_topic(i,10)))

In [33]:
t = plots['Plot'][15629]

In [34]:
print(assign_topic_dist(t))
show_topic_dist(t)

[0.0, 0.3418, 0.419, 0.0, 0.0, 0.0, 0.0, 0.2353]

Score: 0.4180999994277954
Topic 2: 0.010*"kill" + 0.009*"ship" + 0.009*"attack" + 0.008*"soldier" + 0.007*"battle" + 0.007*"escape" + 0.007*"destroy" + 0.007*"use" + 0.007*"war" + 0.007*"power"

Score: 0.3411000072956085
Topic 1: 0.009*"car" + 0.008*"tell" + 0.008*"house" + 0.007*"room" + 0.007*"night" + 0.006*"leave" + 0.006*"find" + 0.006*"hotel" + 0.006*"apartment" + 0.006*"home"

Score: 0.23690000176429749
Topic 7: 0.019*"murder" + 0.018*"police" + 0.013*"kill" + 0.011*"killer" + 0.010*"crime" + 0.010*"case" + 0.009*"prison" + 0.009*"officer" + 0.008*"criminal" + 0.008*"arrest"


In [35]:
plots.iloc[15629]

Release Year                                                     2009
Title                                                  Fast & Furious
Origin/Ethnicity                                             American
Director                                                   Justin Lin
Cast                Vin Diesel, Paul Walker, Michelle Rodriguez, J...
Genre                                                          action
Wiki Page           https://en.wikipedia.org/wiki/Fast_%26_Furious...
Plot                Five years after the first film, Dominic Toret...
Name: 15629, dtype: object

In [36]:
# plots[(plots['Release Year']>=1990) & (plots['Release Year']<=2018) & (plots['Origin/Ethnicity']=='American') & 
#       (plots['Genre']=='mystery')][0:50]

#plots[plots['Title'].str.contains('Avenger')]

In [37]:
# drama/bio: 12361, 15935, 15410, 15776, 12307, 12191
# comedy: 12353, 14591, 14672, 14734
# action: 14792, 15127, 15629, 15158, 14852, 12280
# thriller: 14872, 15107, 12552, 13464
# family: 15663, 15948, 12232
# fantasy: 14667, 15405, 15225, 14303
# adventure: 14696, 15737
# crime: 15178, 13917, 15035, 16238, 8795
# Sci-fi: 15855, 9428, 10298, 10322, 16301, 13509
# mystery: 16112, 15582

In [38]:
# t0: drama, 
# t1: bio-drama, comedy, action, family, fantasy, adventure, crime, sci-fi, mystery
# t2: drama, comedy, action, family, fantasy, sci-fi, 
# t3: drama, action
# t4:

### Recommendation system

In [39]:
prob_vectors = plots['Plot'].map(assign_topic_dist)

In [87]:
prob_vectors[15629]

[0.0, 0.3407, 0.4187, 0.0, 0.0, 0.0, 0.0, 0.2367]

In [88]:
def find_similar(movie, vectors):
    similarity = []

    for i, v in enumerate(vectors):
        s = distance.jensenshannon(assign_topic_dist(movie), v)

        if len(similarity) < 5:
            similarity.append([i,s])

        else:
            case = []
            new = [i,s]

            for ind, j in enumerate(similarity):
                case.append(j[1])

            if new[1] < max(case):
                similarity[case.index(max(case))] = new
    
    similarity = sorted(similarity, key = lambda x: -1*x[1])
    return similarity

In [89]:
x = find_similar(t, prob_vectors)
x

[[11021, 0.030193520778288133],
 [15281, 0.029971609453100387],
 [9353, 0.02088843903844027],
 [7040, 0.019877648197289793],
 [15629, 0.0006034568891839021]]

In [93]:
for i in x:
    print(plots['Title'][i[0]])

The Naked Gun: From the Files of Police Squad!
The Mother of Tears
The Domino Principle
I Bury the Living
Fast & Furious
